## Example Analytical Functions

This notebook walks you through how to set up and run the analytical functions for melting and freezing in this library. We then test the numerical melting/freezing case and compare those results to the analytical solutions. Compare figures to those in the supplementary material of our manuscript (https://www.cambridge.org/core/journals/annals-of-glaciology/article/avoiding-slush-for-hotpoint-drilling-of-glacier-boreholes/6FDD0BE8243F1C625E0FAD398E8B5734).

Note: The freezing case is only an 'approximate' solution, so the numerical results do not fit the analytical results precisely (with numerical being a closer representation of freezing in a glacier borehole).

In [ ]:
### Import libraries ###

# Basic libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# If the cylindricalstefan library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '/home/fenics/shared/'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)
    
# Import functions and constants from the cylindricalstefan library
from cylindricalstefan.lib.instantaneous_mixing_model import *
from cylindricalstefan.lib.analytical_pure_solution import *
from cylindricalstefan.lib.constants import constantsHotPointDrill
const = constantsHotPointDrill()

## Melting Model

Here, we compare two melting solutions presented in the manuscript supplementary material.
- Heat flux is supplied at the hole wall as it moves outward
- Heat flux is supplied at the hole center

Then, we test the first solution (which is more realistic in our borehole drilling case) against our numerical solution.

In [ ]:
### Compare melting solutions ###

# Set up constants
R0 = .04                                 # characteristic radius (m)
rs = np.linspace(.04,1,1000)               # radial distance (m)
Q_melt = 2500.                           # heat flux (W)
ts = np.arange(1,1*3600.,.1)             # time (sec)
Sts = np.linspace(.01,.5,10)**2.         # Stefan number

# Initialize figure
plt.figure(figsize=(4,3))

for St in Sts:
    # Nondimensionalized constants
    T_inf = -St*const.L/const.ci         # bulk ice temperature
    Lv = const.L*const.rhoi              # latent heat of fusion
    t0 = R0**2.*Lv/(const.ki*abs(T_inf)) # characteristic time
    
    # Analytical hole radius for flux at hole wall
    T,lam,R_melt,t_melt = analyticalMelt(rs,T_inf,Q_melt)
    R = 2.*lam*np.sqrt(const.alphai*ts)
    w, = plt.plot(ts/t0,R/R0,'k')
    
    # Analytical hole radius for flux at hole center
    T,lam,R_melt,t_melt = analyticalMelt(rs,T_inf,Q_melt,fluxLoc='Center')
    R = 2.*lam*np.sqrt(const.alphai*ts)
    c, = plt.plot(ts/t0,R/R0,'k:')
    
plt.ylim(0,1.)
plt.xlim(0,.1)
plt.legend([w,c],['Wall','Center'],loc=4)
plt.ylabel('R$^*$')
plt.xlabel('t$^*$')

plt.tight_layout()

In [ ]:
### Test the numerical model against the analytical melting solution ###

# Import the numerical model
mod = instantaneous_mixing_model()

# Set the constants
mod.Q_wall = 2500.   # supplied heat flux
mod.R_melt = 0.01    # initial melt radius (cannot start at 0 for numerical reasons)
mod.T_inf = -20.     # bulk ice temperature
mod.Tf_last = mod.Tf # freezing temperature

# Initialize the numerical model to the analytical solution for initial radius
T,lam,R_melt,t_init = analyticalMelt(np.linspace(0,1,100),-20.,2500.,R_target=mod.R_melt)

# Calculate the analytical solution
T,lam,R_melt,t_melt = analyticalMelt(np.linspace(0,1,100),-20.,2500.,R_target=0.04)

# Set time array for model
ts = np.linspace(t_init,t_melt,100)
mod.dt = np.mean(np.gradient(ts))/10.
mod.t_final = t_melt

# model setup
mod.log_transform()
mod.get_domain()
mod.get_initial_conditions(data_dir='../data/')
mod.save_times = mod.ts[::5]
mod.get_boundary_conditions()

# model run
mod.run(data_dir='../data/')
t0_melt = mod.t0

# Figure setup
plt.figure(figsize=(3.5,3))
ax1 = plt.subplot(211)
plt.xlim(0,2.5)
plt.ylabel('T$^*$');
plt.xlabel('r$^*$');
plt.text(0.01,0.8,'(a)',transform=ax1.transAxes)
ax2 = plt.subplot(212)
plt.ylabel('R$^*$')
plt.xlabel('t$^*$')
plt.text(0.01,0.8,'(b)',transform=ax2.transAxes)

# Plot numerical results
for i,t in enumerate(mod.save_times[::20]):
    ax1.plot(mod.r_ice_result[i*20,:]/0.04,mod.T_ice_result[i*20,:]/abs(mod.T_inf),'r:',zorder=1)
ax2.plot(mod.save_times*mod.t0/t0_melt,mod.r_ice_result[:,mod.ice_idx_wall]/.04,'r:',zorder=1,lw=2)

# Plot Analytical results
rs_melt = np.linspace(0,.2,1000)
R_analytical = np.empty_like(mod.save_times[::20])
for i,t in enumerate(mod.save_times[::20]*mod.t0):
    T,lam,R_melt,t_melt = analyticalMelt(rs_melt,mod.T_inf,mod.Q_wall,target='Time',t_target=t)
    R_analytical[i] = R_melt
    ax1.plot(rs_melt/.04,T/abs(mod.T_inf),'k',alpha=0.5,zorder=0);

# Plot radius through time
T,lam,R_melt,t_melt = analyticalMelt(rs_melt,mod.T_inf,mod.Q_wall,target='Time',t_target=mod.save_times[-1]*mod.t0)
R_analytical = np.append(R_analytical,R_melt)
ax2.plot(np.append(mod.save_times[::20],mod.save_times[-1])*mod.t0/t0_melt,R_analytical/.04,'k',zorder=0,lw=2)

plt.tight_layout()

## Freezing Model

Here, we test the numerical model against an approximate analytical freezing solution (based on Crepeau & Siahpush (2008)).

In [ ]:
# Import the model
mod = instantaneous_mixing_model()

# set the timing
mod.R_melt = .04       # melt-out radius
mod.T_inf = -20.       # bulk ice temperature
mod.t_final = 14000.   # final time to end the sumulation
mod.Tf_last = mod.Tf   # freezing point
mod.R_inf = .045       # end of domain (needs to be small-ish for the analytic solution)

# model setup
mod.log_transform()
mod.get_domain()
mod.get_initial_conditions(data_dir='../data/')
mod.u0_i.vector()[:] = -1.
mod.save_times = mod.ts[::50]
mod.get_boundary_conditions()

# model run
mod.run(data_dir='../data/')

# clip the times
mod.save_times = mod.save_times[:len(mod.r_ice_result)]

# Figure setup
plt.figure(figsize=(3.5,3))
ax1 = plt.subplot(211)
plt.xlim(0,1.1)
plt.ylabel('T$^*$');
plt.xlabel('r$^*$');
plt.text(0.01,0.1,'(a)',transform=ax1.transAxes)
ax2 = plt.subplot(212)
plt.ylabel('R$^*$')
plt.xlabel('t$^*$')
plt.text(0.01,0.1,'(b)',transform=ax2.transAxes)

# Plot numerical results
for i,t in enumerate(mod.save_times[1:]):
    i += 1
    ax1.plot(mod.r_ice_result[i,:]/mod.R_inf,mod.T_ice_result[i,:]/abs(mod.T_inf),'r:',zorder=1)
ax2.plot(mod.save_times,mod.r_ice_result[:,mod.ice_idx_wall]/mod.R_inf,'r:',zorder=1)

# Calculate analytical results
r0 = mod.R_inf
T_inf = mod.T_inf
Q_sol = 0.  # heat flux in the solution as the hole freezes (in our case this is always 0)
Ts,Rs,rs,ts = analyticalFreeze(r0,T_inf,Q_sol,verbose=False)
ts += mod.save_times[0]*mod.t0

# Plot analytical results
for i,t in enumerate(ts[::20]):
    ax1.plot(rs/r0,Ts[i*20]/abs(mod.T_inf),'k',alpha=0.5,zorder=0)
ax2 = plt.plot(ts/mod.t0,Rs/r0,'k',zorder=0)
    
plt.tight_layout()